**Bronze Layer**

In [0]:
from pyspark.sql import functions as F

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.bronze")

In [0]:
# BRONZE: Raw ingestion
raw = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)

In [0]:
raw.withColumn("ingestion_ts", F.current_timestamp()) \
   .write.format("delta").mode("overwrite").saveAsTable("workspace.bronze.events")

**Silver Layer**

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.silver")

In [0]:
# SILVER: Cleaned data
bronze = spark.read.table("workspace.bronze.events")

In [0]:
silver = bronze.filter(F.col("price") > 0) \
    .filter(F.col("price") < 10000) \
    .dropDuplicates(["user_session", "event_time"]) \
    .withColumn("event_date", F.to_date("event_time")) \
    .withColumn("price_tier",
        F.when(F.col("price") < 10, "budget")
         .when(F.col("price") < 50, "mid")
         .otherwise("premium"))


In [0]:
silver.write.format("delta").mode("overwrite").saveAsTable("workspace.silver.events")

**Gold Layer**


In [0]:
from pyspark.sql import functions as F
        

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.gold")


In [0]:
# GOLD: Aggregates
silver = spark.read.format("delta").table("workspace.silver.events")


In [0]:
product_perf = silver.groupBy("product_id", "brand") \
    .agg(
        F.countDistinct(F.when(F.col("event_type")=="view", "user_id")).alias("views"),
        F.countDistinct(F.when(F.col("event_type")=="purchase", "user_id")).alias("purchases"),
        F.sum(F.when(F.col("event_type")=="purchase", F.col("price").cast("double"))).alias("revenue")
    ).withColumn("conversion_rate", F.try_divide(F.col("purchases"), F.col("views"))*100)


In [0]:
product_perf.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.Products")